<img style="float: right; height:90px" src="dove_air.png"> 

 # Sierra Leone Medical Resources Distribution Analysis

<div style="text-align: right">Author: Phil Li</div>
<div style="text-align: right"><b>Engineering and Data Analyst</b></div>

In [1]:
import pandas as pd
#%matplotlib inline
%matplotlib widget
from math import radians, cos,sin,asin,sqrt
# pd.set_option('display.max_columns',30)
from ipywidgets import HTML
import os,sys
import geopandas as gpd
import ipyvuetify as v
import psycopg2
import os,sys 
import urllib.parse as urlparse
import warnings
def ignore_warn(*args,**kwargs):
    pass
warnings.warn=ignore_warn

In [2]:
for root, dirs, files in os.walk("/opt/conda/lib/python3.8/site-packages"):
    for f in files:
        os.chmod(os.path.join(root, f), 0o755)

In [3]:
import mapclassify 
from ipyleaflet import Choropleth,Map,GeoJSON,basemaps,ScaleControl,LayersControl, FullScreenControl,MeasureControl,AwesomeIcon,Marker,Popup,Circle,LayerGroup,LayersControl,MarkerCluster,SearchControl,GeoData,WidgetControl
from branca.colormap import linear

In [4]:
url=urlparse.urlparse(os.environ['DATABASE_URL'])
dbname=url.path[1:]
user=url.username
password=url.password
host=url.hostname
port=url.port

conn=psycopg2.connect(dbname=dbname,
                    user=user,
                    password=password,
                    host=host,
                    port=port)

In [5]:
# conn=psycopg2.connect(host='localhost',database='sierra_geo',user='postgres')
population_cal_df=pd.read_sql("SELECT * from population_cal",conn)
district_info_df=gpd.read_postgis("SELECT district_name,region_name,population,polygon AS geometry FROM district",conn,geom_col='geometry',crs=4326)
medical_gdf=gpd.read_postgis("SELECT district,name,type AS facility_type, latitude,longitude,point AS geometry FROM medical",conn,geom_col='geometry',crs=4326)
eez_shape_df=gpd.read_postgis("SELECT line_name,line AS geometry FROM eez",conn,geom_col='geometry',crs=4326)
conn.close()

In [6]:
## Choropleth

In [7]:
features=[]
for i in district_info_df.district_name.values:
    single_json=gpd.GeoSeries(district_info_df[district_info_df['district_name']==i].geometry.values).__geo_interface__
    single_json['features'][0]['id']=i
    single_json['features'][0]['properties']['name']=i
    features.append(single_json['features'][0])
district_json={"type":"FeatureCollection","features":features}

In [8]:
choro_data={}
for i,key in enumerate(district_info_df.population.sort_values().index.values):
    color=5+(i//2)*0.4
    choro_data[district_info_df.iloc[key].district_name]=color

In [9]:
layer_choro=Choropleth(
geo_data=district_json,
choro_data=choro_data,
colormap=linear.YlOrRd_04,
style={'fillOpacity': 0.8, 'dashArray': '5, 5'},
key_on='id',
name='Choropleth')

In [10]:
## Site Coverage

In [11]:
circle_1=Circle(location=(medical_gdf[medical_gdf['name']=='Kindoya Hospital'].latitude.values[0],medical_gdf[medical_gdf['name']=='Kindoya Hospital'].longitude.values[0]),
             radius=150000,
             color='aqua',
             stroke=False,
             fill_opacity=0.5,
               name='Kindoya Hospital, Bo')
circle_2=Circle(location=(medical_gdf[medical_gdf['name']=='Port Loko Government Hospital'].latitude.values[0],medical_gdf[medical_gdf['name']=='Port Loko Government Hospital'].longitude.values[0]),
             radius=150000,
             color='tomato',
             stroke=False,
             fill_opacity=0.5,
               name='Port Loko Government Hospital, Port Loko')
circle_group=LayerGroup(name='Site Coverage',layers=(circle_1,circle_2))

In [12]:
## Population Calculation

In [13]:
def distance_cal(hospital_n,section_n):
    """"
    calculate the distance between two points on the earth
    """
    #conver decimal degrees to radians
    sec=population_cal_df[population_cal_df['section']==section_n]
    hosp=medical_gdf[medical_gdf.name==hospital_n]
    lon1,lat1,lon2,lat2=map(radians,[sec.longitude,
                            sec.latitude,
                            hosp.longitude,
                            hosp.latitude])
    #haversing formula
    dlon=lon2-lon1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c=2*asin(sqrt(a))
    r=6371
    return round(c*r,2)

site_population={}

def population_cal(hospital_n):
    """"
    calculate the population covered within the circle
    """
    pop={}
    for hosp in hospital_n:
        pop[hosp]=0
        for section in population_cal_df.section:
            dis=distance_cal(hosp,section)
            if dis>=145 and dis<153:
                pop[hosp]=pop[hosp]+0.5*population_cal_df[population_cal_df['section']==section].population.values[0]
            elif dis<145:
                pop[hosp]=pop[hosp]+population_cal_df[population_cal_df['section']==section].population.values[0]
            else:
                continue
        pop[hosp]=round(pop[hosp])
        
    site_population.clear()
    for loc,population in pop.items():
        site_population[f'{loc}, {medical_gdf[medical_gdf["name"]==loc].district.values[0]}']=population 
population_cal(['Kindoya Hospital','Port Loko Government Hospital'])

In [14]:
# Ipyvuetify
chip_1=v.Chip(
              class_='ma-1',
              color='success',
              outlined=True,
              x_small=True,
              children=[v.Icon(left=True,children=['mdi-account-group']),f'{list(site_population.keys())[0]}: {site_population[list(site_population.keys())[0]]} (Population)'
              ])
chip_2=v.Chip(prepend_icon='mdi-hospital-box',
              class_='ma-1',
              color='green',
              x_small=True,
              text_color="white",
              children=[v.Icon(left=True,children=['mdi-account-group']),f'Port Loko Govt Hospital: {site_population[list(site_population.keys())[1]]} (Population)'])

box=v.Html(tag='div', class_='d-flex flex-column', children=[
        chip_1,
        chip_2
    ])

widget_control = WidgetControl(widget=box, position='bottomright')

In [15]:
## Hospitals & Clinics

In [18]:
medical_gdf=medical_gdf[(medical_gdf['district']=='Bo') | (medical_gdf['district']=='Kenema') ].reset_index(drop=True)

In [19]:
icon1 = AwesomeIcon(
    name='hospital-o',
    marker_color='blue',
    icon_color='black',
    spin=False    
)

hospital=[]
for i,key in enumerate(medical_gdf[medical_gdf['facility_type']=='HOSPITAL'].index.values):
    series=medical_gdf.iloc[key]
    marker=Marker(icon=icon1, draggable=False,opacity=0.8,location=(series.latitude,series.longitude))
    message=HTML()
    message.value=f'<b>{medical_gdf.iloc[key,1]}</b>'
    marker.popup=message
    hospital.append(marker)
hospital=tuple(hospital)
marker_cluster=MarkerCluster(name='Hospitals',markers=hospital)

In [20]:
## EEZ

In [21]:
geo_data = GeoData(geo_dataframe = eez_shape_df,
                   style={'color': 'red', 'fillColor': '#3366cc', 'opacity':0.4, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'white' , 'fillOpacity': 0.2},
                   name = 'eez')

In [22]:
## Search Control

In [23]:
district_SL = GeoJSON(data=district_json,style={
        'opacity': 0.5, 'dashArray': '9', 'fillOpacity': 0.05, 'weight': 1},
         hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.1
    })
layer_group = LayerGroup(layers=(district_SL,))
marker_search = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='darkgreen'))
search_control=SearchControl(position="topleft",
#   url='https://nominatim.openstreetmap.org/search?format=json&q={s}',
  layer=layer_group,
  zoom=10,
  property_name='name',
  marker=marker_search,
  found_style={'fillColor': '#3f0', 'color': '#0f0'})

In [24]:
## Measure Control

In [25]:
measure = MeasureControl(
    position='topleft',
    active_color = 'blue',
    primary_length_unit = 'kilometers',
    primary_area_unit='sqmeters'
)
measure.completed_color = 'green'

In [26]:
## MAP

## Analysis Examples
Some analysis results are illustrated below. This interactive map serves as a standalone dashboard for exploring the medical resources distribution and Dove Air's urgent medicine delivery service range in Sierra Leone.

In [27]:
center =[7.961902,-11.720791]
zoom=7
# Layer Control
layer_control = LayersControl(position='topright',hide=True)

m=Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)

m.add_control(FullScreenControl())
m.add_control(search_control)
m.add_layer(circle_group)
m.add_layer(layer_choro)
m.add_layer(marker_cluster)
m.add_layer(geo_data)
m.add_control(widget_control)
m.add_control(layer_control)
m.add_control(measure)

m

Map(center=[7.961902, -11.720791], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

Features:
* `Measure` (Top Left) - measure distances and areas by adding points to the map
* `Search` (Top Left) - districts in Sierra Leone could be found through the search engine (eg: `Bo`, `Kenema`, `Port Loko`, `Western Area Urban`)
* `Layer Selection` (Top Right) - select or deselect layers on top of the map
* `Service Coverage` (Bottom Right) - population coverage of each service center

Layers:
* `Site Coverage` - circles with the centers at Dove Air service centers and 150km radius (delivery range)
* `Choropleth` - total population of each district (darker color represents higher population)
* `Hospitals` - hospitals within Sierra Leone (data is incomplete)
* `eez` - exclusive economic zone 

Notes:
* Please see hospitals in `Bo` or `Kenema` for a city-level demo
* It is advised to deselect `Site Coverage` and `Choropleth` layers for better visibility at city levels
* <p style='color:red'> It may take a minute or two for the display to first appear...</p>